### 矩阵分解

尽管 user-based 和 item-based 的协同过滤算法简单直观，矩阵分解往往更有效，因为它能够发掘用户和物品之间的潜在特征。

#### Basic ideas

矩阵分解，顾名思义，就是把一个矩阵分解为两个（或更多）矩阵，使得它们的乘积等于原矩阵。从应用的角度来讲，矩阵分解可以发掘用户和物品之间的潜在特征，可以应用到协同过滤的排名预测中。

在推荐系统，比如 Netflix 或 MovieLens 中，有一组用户和物品（比如电影）。每个用户都对一些物品打了分数，我们要预测用户对没评分的物品打的分，这样就可以做推荐。比如有5个用户，10个物品，评分是1到5之间的整数，可以用矩阵表示如下：

|    /    |  D1 | D2 | D3 | D4 |
| ------  | --- | --- | --- | --- |
| U1 |  5 | 3 | - | 1 |
| U2 |  4 | - | - | 1 |
| U3 |  1 | 1 | - | 5 |
| U4 |  1 | - | - | 4 |

其中，"-" 表示没有打分的项。使用矩阵分解解决问题的直觉就是应该有潜在的特征来决定用户如何对物品打分，比如，两个用户如果都喜欢某个电影的演员，或者两个用户都喜欢动作片，那么他们都应该对该电影打高分。因而，如果能够发现这些潜在特征，我们就能够对特定用户和物品做出准确预测。

为了试图找出不同特征，做如下假设：特征的数目要小于用户和物品的数目（因为假设每个用户对应一个单独的特征是不合理的），如果非要每个用户对应一个单独的特征，那么做推荐是没有意义的，因为用户将对其他用户评分的物品都不感兴趣。

#### The mathematics for matrix factorization

用户集合为 $U$, 物品集合为 $D$，$R$ 为 $|U|\times|D|$ 的矩阵，表示用户对物品的评分，我们想找出 $K$ 潜在特征。我们的目标是找出两个矩阵 $P(|U|\times K 矩阵)$ 和 $Q(|D|\times K 矩阵)$，使它们的乘积近似等于 $R$。
$$R\approx P\times Q^T=\hat{R}$$
这样，$P$ 的每一行表示用户和特征的关联强度，同样，$Q$ 的每一行表示物品和特征的关联强度。为得到 $u_i$ 对物品 $d_j$ 的预测评分，只需要计算 $u_i$ 和 $d_j$ 对应向量的点积：
$$\hat{r}_{ij}=p_iq^T_j=\sum^k_{k=1}p_{ik}q_{kj}$$
接下来的问题就是如何找到 $P$ 和 $Q$，这里用到经典的梯度下降法。预测评分和真实评分之间的 error 为：
$$e^2_{ij}=(r_{ij}-\hat{r}_{ij})^2=(r_{ij}-\sum^K_{k=1}p_{ik}q_{kj})^2$$
为了最小化 error，我们需要知道当前值的梯度，即需要在哪个方向改变 $p_{ik}$ 和 $q_{kj}$ 的值，进行求导：
$$\frac{\partial}{\partial p_{ik}}e^2_{ij}=-2(r_{ij}-\hat{r}_{ij})(q_{kj})=-2e_{ij}q_{kj}$$
$$\frac{\partial}{\partial q_{kj}}e^2_{ij}=-2(r_{ij}-\hat{r}_{ij})(p_{ik})=-2e_{ij}p_{ik}$$
得到梯度后，更新函数为：
$$p'_{ik}=p_{ik}-\alpha\frac{\partial}{\partial p_{ik}}e^2_{ij}=p_{ik}+2\alpha e_{ij}q_{kj}$$
$$q'_{kj}=q_{kj}-\alpha\frac{\partial}{\partial q_{kj}}e^2_{ij}=q_{kj}+2\alpha e_{ij}p_{ik}$$
其中，$\alpha$ 为学习率。

设 $T$ 为元组集合 $(u_i, d_j, r_{ij})$，包含所有的 user-item 对及评分，我们需要最小化训练集中每个 $e_{ij}$: 

$E=\sum_{(u_i,d_j,r_{ij})\in T}e_{ij}=\sum_{(u_i,d_j,r_{ij})\in T}(r_{ij}-\sum^K_{k=1}p_{ik}q_{kj})^2$

#### Regularization

为避免过拟合，可加入正则项：
$$e^2_{ij}=(r_{ij}-\sum^K_{k=1}p_{ik}q_{kj})^2+\frac{\beta}{2}\sum^{K}_{k=1}(\|P\|^2+\|Q\|^2)$$
新的更新函数为：
$$p'_{ik}=p_{ik}-\alpha\frac{\partial}{\partial p_{ik}}e^2_{ij}=p_{ik}+\alpha(2e_{ij}q_{kj}-\beta p_{ik})$$
$$q'_{kj}=q_{kj}-\alpha\frac{\partial}{\partial q_{kj}}e^2_{ij}=q_{kj}+\alpha(2e_{ij}p_{ik}-\beta q_{kj})$$

#### Implementation in Python

In [8]:
#!/usr/bin/python
#
# Created by Albert Au Yeung (2010)
#
# An implementation of matrix factorization
#
try:
    import numpy
except:
    print("This implementation requires the numpy module.")
    exit(0)

###############################################################################

"""
@INPUT:
    R     : a matrix to be factorized, dimension N x M
    P     : an initial matrix of dimension N x K
    Q     : an initial matrix of dimension M x K
    K     : the number of latent features
    steps : the maximum number of steps to perform the optimisation
    alpha : the learning rate
    beta  : the regularization parameter
@OUTPUT:
    the final matrices P and Q
"""
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T

###############################################################################

if __name__ == "__main__":
    R = [
         [5,3,0,1],
         [4,0,0,1],
         [1,1,0,5],
         [1,0,0,4],
         [0,1,5,4],
        ]

    R = numpy.array(R)

    N = len(R)
    M = len(R[0])
    K = 2

    P = numpy.random.rand(N,K)
    Q = numpy.random.rand(M,K)

    nP, nQ = matrix_factorization(R, P, Q, K)
    
    print(numpy.dot(nP, nQ.T))

[[5.03425208 2.83185449 5.07438341 0.9974816 ]
 [3.93635679 2.21817193 4.1174538  0.99683758]
 [1.10586641 0.70745206 4.39022404 4.96430059]
 [0.9484875  0.60165871 3.56925553 3.97361162]
 [2.38339376 1.4048369  4.86285502 4.03664462]]
